In [ ]:
!pip install hyperas   #加上 ! 執行pip指令。

In [ ]:
# 在這邊導入Hyperas。
from hyperopt import hp
from hyperopt import Trials, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

def prepare_data():
    """ 
    準備資料
    """
    # 在這邊導入外部資料庫。
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import KFold
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Activation, Dropout

    #將train.csv讀取到pandas的DataFrame。
    train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
    train_x = train.drop(['label'], axis=1)   # 從train取出圖像資料
    train_y = train['label']     # 從train取出正確答案標籤

    # 將train資料分為訓練資料與驗證資料。
    kf = KFold(n_splits=4, shuffle=True, random_state=123)
    tr_idx, va_idx = list(kf.split(train_x))[0]
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    # 圖像像素值除以 255.0，限制在0 ~ 1.0範圍內，並轉換為numpy.array。
    tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

    # 將正確答案以One-hot encoding呈現。
    tr_y = to_categorical(tr_y, 10)
    va_y = to_categorical(va_y, 10)

    return tr_x, tr_y, va_x, va_y

def create_model(tr_x, tr_y):
    """ 
    生成模型
    """
    # 生成Sequential object。
    model = Sequential()

    # 配置第 1 層，神經元數量設為500或784。
    model.add(Dense(
        {{choice([500, 784])}},
        input_dim=tr_x.shape[1],
        activation='relu'
    ))
    model.add(Dropout(0.4))

    # 從0,1,2當中探尋要新增的層的數量。
    if {{choice(['none', 'one', 'two'])}} == 'none':
        #若選到none則不新增層。
        pass
    elif {{choice(['none', 'one', 'two'])}} == 'one':
        # 若選到one 則配置第2層、探尋神經元數量。
        model.add(Dense(
            {{choice([100, 200])}},
            activation='relu'
        ))

    elif {{choice(['none', 'one', 'two'])}} == 'two':
        # 若選到two則配置第3層、探尋各層神經元數量。
        model.add(Dense(
            {{choice([100, 200])}},
            activation='relu'
        ))
        model.add(Dense(
            {{choice([25, 50])}},
            activation='relu'
        ))

    # 配置輸出層。
    # 類別數量是既定的，故無須探尋Unit數量。
    model.add(Dense(10, activation="softmax"))

    # 編譯模型。
    # 嘗試使用優化器Adam 與 RMSprop。
    model.compile(loss="categorical_crossentropy",
    optimizer={{choice(['adam', 'rmsprop'])}},
    metrics=["accuracy"])

    epoch = 10 # 學習次數。
    batch_size = 100  # 小批次的尺寸。

    # 進行訓練
    result = model.fit(tr_x,
                       tr_y,
                       epochs=epoch,
                       batch_size=batch_size,
                       validation_data=(va_x, va_y),
                       verbose=0)

    # 輸出探尋時的準確度。
    validation_acc = np.amax(result.history['val_accuracy'])
    print('Accuracy in search:', validation_acc)

    return {'loss': -validation_acc, 
            'status': STATUS_OK, 
            'model': model}

# 執行探尋
nb_name = '__notebook_source__'
best_run, best_model = optim.minimize(model=create_model,
                                      data=prepare_data,
                                      algo=tpe.suggest,
                                      max_evals=20,
                                      eval_space=True,
                                      notebook_name=nb_name,
                                      trials=Trials()
                                     )


In [ ]:
# 輸出準確度最好的模型。
print(best_model.summary())
# 輸出準確度最好的超參數。
print(best_run)

# 使用驗證資料來檢驗模型。
_, _, va_x, va_y = prepare_data()
val_loss, val_acc = best_model.evaluate(va_x, va_y)
print("val_loss: ", val_loss) # 輸出損失。
print("val_acc: ", val_acc) # 輸出準確度。
